# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite')

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [10]:
# Your code here
q = """ SELECT *
        FROM offices;
       """

pd.read_sql(q, conn)

,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
1,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
2,3,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA
3,4,Paris,+33 14 723 4404,43 Rue Jouffroy D'abbans,,,France,75017,EMEA
4,5,Tokyo,+81 33 224 5000,4-1 Kioicho,,Chiyoda-Ku,Japan,102-8578,Japan
5,6,Sydney,+61 2 9264 2451,5-11 Wentworth Avenue,Floor #2,,Australia,NSW 2010,APAC
6,7,London,+44 20 7877 2041,25 Old Broad Street,Level 7,,UK,EC2N 1HN,EMEA
7,27,Boston,+1 977 299 8345,105 Cambridge Street,,MA,USA,02331,NA


In [3]:
# Your code here
q = """ SELECT lastName, firstName
        FROM employees AS e
        JOIN offices AS o
        ON e.officeCode = o.officeCode 
        WHERE city = 'Boston';
       """

pd.read_sql(q, conn)

,lastName,firstName
0,Firrelli,Julie
1,Patterson,Steve


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [4]:
# Note that COUNT(*) is not appropriate here because
# we are trying to count the _employees_ in each group.
# So instead we count by some attribute of an employee
# record. The primary key (employeeNumber) is a 
# conventional way to do this
q = """
SELECT
    o.officeCode,
    o.city,
    COUNT(e.employeeNumber) AS n_employees
FROM offices AS o
LEFT JOIN employees AS e
    USING(officeCode)
GROUP BY officeCode
HAVING n_employees = 0
;
"""
pd.read_sql(q, conn)

,officeCode,city,n_employees
0,27,Boston,0


## Write 3 questions of your own and answer them

In [11]:
# Answers will vary

# Example question: 
"""
How many employees are there per office?
"""
q = """
SELECT
    o.officeCode,
    o.city,
    COUNT(e.employeeNumber) AS n_employees
FROM offices AS o
LEFT JOIN employees AS e
    USING(officeCode)
GROUP BY officeCode;
"""
pd.read_sql(q, conn)

,officeCode,city,n_employees
0,1,San Francisco,6
1,2,Boston,2
2,3,NYC,2
3,4,Paris,5
4,5,Tokyo,2
5,6,Sydney,4
6,7,London,2
7,27,Boston,0


In [12]:

# Example question: 
"""
How many customers are there per office?
"""

# Example question answer:
q = """
SELECT
    o.officeCode,
    o.city,
    COUNT(c.customerNumber) AS n_customers
FROM offices AS o
JOIN employees AS e
    USING(officeCode)
JOIN customers AS c
    ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY officeCode
;
"""
pd.read_sql(q, conn)

,officeCode,city,n_customers
0,1,San Francisco,12
1,2,Boston,12
2,3,NYC,15
3,4,Paris,29
4,5,Tokyo,5
5,6,Sydney,10
6,7,London,17


In [14]:
"""
Question 2
"""



'\nQuestion 2\n'

In [ ]:
"""
Question 3
"""

# Your code here

## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [16]:
# Your code here

# Example question: 
"""
How many customers are there per office?
"""

# Example question answer:
q = """
SELECT
    productName,
    lastName,
    firstName
FROM employees AS e
JOIN customers AS c
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders 
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
JOIN products
    USING(productCode);
"""
pd.read_sql(q, conn)

,productName,lastName,firstName
0,1958 Setra Bus,Jennings,Leslie
1,1940 Ford Pickup Truck,Jennings,Leslie
2,1939 Cadillac Limousine,Jennings,Leslie
3,1996 Peterbilt 379 Stake Bed with Outrigger,Jennings,Leslie
4,1968 Ford Mustang,Jennings,Leslie
...,...,...,...
2991,1954 Greyhound Scenicruiser,Gerard,Martin
2992,1950's Chicago Surface Lines Streetcar,Gerard,Martin
2993,Diamond T620 Semi-Skirted Tanker,Gerard,Martin
2994,1911 Ford Town Car,Gerard,Martin


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [22]:
# Your code here
q = """
SELECT
    SUM(quantityOrdered) AS total_products_sold,
    lastName,
    firstName
FROM employees AS e
JOIN customers AS c
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders 
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
GROUP BY firstName, lastName
ORDER BY total_products_sold DESC;
"""
pd.read_sql(q, conn)

,total_products_sold,lastName,firstName
0,14231,Hernandez,Gerard
1,11854,Jennings,Leslie
2,9290,Castillo,Pamela
3,8205,Bott,Larry
4,7486,Jones,Barry
5,7423,Vanauf,George
6,6632,Marsh,Peter
7,6246,Fixter,Andy
8,6186,Bondur,Loui
9,5561,Patterson,Steve


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [24]:
# Your code here
# Recall that HAVING is used for filtering after an aggregation
q = """
SELECT firstName, lastName, COUNT(productCode) as different_products_sold
FROM employees AS e
JOIN customers AS c
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
GROUP BY firstName, lastName
HAVING different_products_sold > 200
ORDER BY lastName
;
"""
pd.read_sql(q, conn)

,firstName,lastName,different_products_sold
0,Larry,Bott,236
1,Pamela,Castillo,272
2,Gerard,Hernandez,396
3,Leslie,Jennings,331
4,Barry,Jones,220
5,George,Vanauf,211


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!